# Tareas

**1) Diseñar nuestro modelo:** (input, output, size, forward pass)

**2) Construir "loss"(perdida) y "optimizer"(optimizador)**

**3) Bucle de entrenamiento:**
 - Forward pass: computar predicción
 - backward pass: gradientes
 - Actulizar los pesos 

In [1]:
import torch
import torch.nn as nn
#import torch.nn.functional as F

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

## Preparamos los datos 

In [2]:
# Cargamos nuestro data set
bc = datasets.load_breast_cancer()

# Guardamos nuestros datos en "x" y nuestro objetivos en "y"
X, y = bc.data, bc.target

In [3]:
# Guardamos el numero datos en "n_features" y nuestro numero de objetivos en "n_samples"
n_samples, n_features = X.shape

# Divimos nuestro datos de entranamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [4]:
# Gurdamos nuestro "StandardScaler" (una especie de normalizacion que aplica en las columnas)
sc = StandardScaler()

# Entrenamos a nuestro "StandardScaler"
X_train = sc.fit_transform(X_train)

# Testeamos a nuestro "StandardScaler"
X_test  = sc.transform(X_test)

In [5]:
# Transformamos los datos de un array numpy a Tensor "float32"
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test  = torch.from_numpy(y_test.astype(np.float32))

In [6]:
# Transformamos los objetivos para que solo tengan una columna 
y_train = y_train.view(y_train.shape[0], 1)
y_test  = y_test.view(y_test.shape[0], 1)

## Modelo

In [7]:
# f = wx + b ,sigmod al final
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        # Volvemos a "LogisticRegression" una seper clase
        super(LogisticRegression, self).__init__()
        # Iniciamos con una capa "lineal" simple que tome nuestros features y devuleva una prediccion
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        # Tomamos la prediccion echa por la capa lineal y se la damos a una capa "sigmoid"
        # Luego Guardamos nuestra nueva prediccion en "y_predicted" y lo retornamos
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

# Guardasmos nuesto modelo en "model"
model = LogisticRegression(n_features)

## Perdida(loss) y Optimizador(optmizer)

In [8]:
learning_rate = 0.01
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Bucle de entrenamiento

In [9]:
num_epocns = 100

for epoch in range(num_epocns):
    
    # Forward pass(pase hacia adelante)
    y_predicted = model(X_train)
    
    # Loss(Perdida)
    loss = criterion(y_predicted, y_train)
    
    # Backward pass(pase hacia atras)
    loss.backward()
    
    # Actulizacion
    optimizer.step()
    
    # Vover a 0 los gradientes
    optimizer.zero_grad()
    
    # Resultados
    if (epoch+1) % 10 == 0:
        print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")

epoch: 10, loss = 0.2145
epoch: 20, loss = 0.1827
epoch: 30, loss = 0.1596
epoch: 40, loss = 0.1425
epoch: 50, loss = 0.1294
epoch: 60, loss = 0.1190
epoch: 70, loss = 0.1107
epoch: 80, loss = 0.1039
epoch: 90, loss = 0.0982
epoch: 100, loss = 0.0933


## Testeo

In [10]:
# relu
# Usamos "torch.no_grad()" para que no se incluya ennnuestro grafico computacional
with torch.no_grad():
    # Hacemos predicciones con nuestros datos de testeo
    y_predicted = model(X_test)
    
    # Redondeamos los resultados
    y_predicted_cls = y_predicted.round()
    
    #Calculamos la precicion de nuestro modelo ((Suma respuestas correctas) / (Suma respuestas incorrectas))
    accuaracy = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    
    #Mostramos la exactitud de nuestro modelo
    print(f"accuracy = {accuaracy:.4f}")

accuracy = 0.8860
